# Classificação de Sentimentos com Word2Vec e Naive Bayes (usando CSV)

Este notebook realiza a classificação de sentimentos (feliz ou triste) com base em textos usando técnicas de **Processamento de Linguagem Natural (PLN)** com **Word2Vec** e **Naive Bayes**. As etapas incluem:
- Leitura da base de dados (CSV)
- Limpeza e tokenização dos textos com remoção de stopwords (português)
- Vetorização com Word2Vec (média dos vetores de palavras)
- Treinamento e avaliação com `GaussianNB`
- Predição com frases novas (`teste.txt`)

In [1]:
# Instalação das bibliotecas necessárias
!pip install -q pandas scikit-learn gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 9.2 MB/s eta 0:00:00


In [32]:
import pandas as pd  # Manipulação de dados com DataFrames (leitura, escrita e organização de tabelas)
import numpy as np  # Biblioteca para operações numéricas e manipulação de arrays
import re  # Expressões regulares para limpeza e pré-processamento de texto
from gensim.models import Word2Vec  # Geração de vetores de palavras a partir do corpo do texto
from sklearn.naive_bayes import GaussianNB  # Classificador Naive Bayes Gaussiano (assume distribuição normal)
from sklearn.model_selection import train_test_split  # Separar os dados em conjuntos de treino e teste
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  # Métricas de avaliação do modelo
from imblearn.over_sampling import SMOTE  # Técnica para balancear classes
from sklearn.preprocessing import LabelEncoder  # Codificação de rótulos (strings para inteiros)
from collections import Counter  # Contagem de frequência de elementos, útil para verificar o balanceamento

## Leitura da base CSV

In [33]:
df = pd.read_csv("FelizOuTriste.csv")  # Carrega o arquivo CSV com frases (texto) e classificador (sentimento) em um DataFrame
df.columns = ['texto', 'sentimento']  # Renomeia as colunas para facilitar a referência no código
df.head(10)  # Exibe as primeiras 10 linhas do DataFrame para inspeção inicial

,texto,sentimento
0,Sorria! Deus acaba de te dar um novo dia e coi...,Feliz
1,Contarte longamente as perigosas coisas do mar...,Feliz
2,Bom dia alegria!,Feliz
3,A amizade desenvolve a felicidade e reduz o so...,Feliz
4,Sorria bonito e deixe os outros quererem desco...,Feliz
5,Quem tentar possuir uma flor verá sua beleza m...,Feliz
6,O amor surge quando aprendemos a admirar as qu...,Feliz
7,O nosso verdadeiro lugar de nascimento é aquel...,Feliz
8,O sábio pode descobrir o mundo sem transpor a ...,Feliz
9,As vezes você perdoa uma pessoa porque a falta...,Triste


## Limpeza de texto e tokenização com stopwords

In [34]:
# Carregar stopwords
with open("stopwords_pt.txt", "r", encoding="ISO-8859-1") as f:  # Abre o arquivo de stopwords com encoding compatível
    stopwords = set(f.read().splitlines())  # Lê e armazena as stopwords em um conjunto para busca rápida

def limpar_texto(texto):  # Função para limpar e tokenizar o texto
    texto = texto.lower()  # Converte para minúsculas
    texto = re.sub(r'[^a-záéíóúãõç\s]', '', texto)  # Remove caracteres que não são letras nem espaços
    palavras = texto.split()  # Separa o texto em palavras
    palavras = [p for p in palavras if p not in stopwords]  # Remove as stopwords da lista
    return palavras  # Retorna a lista de tokens relevantes

df['tokens'] = df['texto'].apply(limpar_texto)  # Aplica a função de limpeza em todas as frases do DataFrame
df.head(10)  # Exibe as primeiras 10 linhas para verificar os tokens gerados

,texto,sentimento,tokens
0,Sorria! Deus acaba de te dar um novo dia e coi...,Feliz,"[sorria, deus, acaba, dar, novo, dia, coisas, ..."
1,Contarte longamente as perigosas coisas do mar...,Feliz,"[contarte, longamente, perigosas, coisas, mar,..."
2,Bom dia alegria!,Feliz,"[bom, dia, alegria]"
3,A amizade desenvolve a felicidade e reduz o so...,Feliz,"[amizade, desenvolve, felicidade, reduz, sofri..."
4,Sorria bonito e deixe os outros quererem desco...,Feliz,"[sorria, bonito, deixe, quererem, descobrir, é..."
5,Quem tentar possuir uma flor verá sua beleza m...,Feliz,"[tentar, possuir, flor, verá, beleza, murchand..."
6,O amor surge quando aprendemos a admirar as qu...,Feliz,"[amor, surge, aprendemos, admirar, qualidades,..."
7,O nosso verdadeiro lugar de nascimento é aquel...,Feliz,"[verdadeiro, lugar, nascimento, é, lançamos, p..."
8,O sábio pode descobrir o mundo sem transpor a ...,Feliz,"[sábio, pode, descobrir, mundo, transpor, port..."
9,As vezes você perdoa uma pessoa porque a falta...,Triste,"[vezes, voc, perdoa, pessoa, falta, faz, vida,..."


## Vetorização com Word2Vec (média dos vetores)

In [36]:
# Gerar sentenças e treinar modelo Word2Vec
sentencas = df['tokens'].tolist()  # Converte a coluna de tokens em uma lista de listas (sentenças)
modelo_w2v = Word2Vec(sentencas, vector_size=100, window=5, min_count=1, workers=4)  # Treina o modelo Word2Vec com vetores de 100 dimensões

# Função para calcular vetor médio da frase
def vetor_frase(tokens):  # Recebe os tokens de uma frase
    vetores = [modelo_w2v.wv[p] for p in tokens if p in modelo_w2v.wv]  # Recupera o vetor de cada palavra presente no vocabulário
    return sum(vetores) / len(vetores) if vetores else [0]*100  # Retorna a média dos vetores ou vetor nulo se estiver vazio

# Aplicar a vetorização
X = df['tokens'].apply(vetor_frase).tolist()  # Aplica a função em cada frase e gera a matriz de vetores
y = df['sentimento']  # Define a variável alvo com os rótulos de sentimento

# Visualizar alguns vetores gerados
print("Exemplo de vetor Word2Vec da primeira frase:")
print(np.round(X[0], 3))  # Mostra o vetor da primeira frase com 3 casas decimais

print("\nFormato da matriz X (frases x dimensões):", np.array(X).shape)  # Mostra o formato da matriz vetorizada

# Visualizar o vetor da palavra "feliz" (se existir no vocabulário)
if "feliz" in modelo_w2v.wv:  # Verifica se a palavra "feliz" está no vocabulário treinado
    print("\nVetor da palavra 'feliz':")
    print(np.round(modelo_w2v.wv["feliz"], 3))  # Mostra o vetor da palavra com 3 casas decimais

Exemplo de vetor Word2Vec da primeira frase:
[-0.003 -0.002 -0.002 -0.    -0.002  0.    -0.001 -0.001 -0.002 -0.
 -0.001 -0.004  0.     0.001 -0.001 -0.005  0.    -0.001 -0.001  0.002
  0.002  0.001  0.001  0.003  0.001 -0.001  0.    -0.001  0.     0.001
 -0.002 -0.     0.    -0.003  0.001  0.001 -0.001 -0.001  0.001 -0.001
 -0.    -0.002  0.001  0.     0.     0.001 -0.001  0.     0.001 -0.002
  0.002 -0.003  0.004 -0.001 -0.    -0.001  0.    -0.001 -0.    -0.001
  0.001 -0.002 -0.001 -0.    -0.001  0.001 -0.002  0.003 -0.002  0.002
 -0.     0.002  0.002 -0.001  0.001  0.001  0.     0.    -0.001 -0.
 -0.002 -0.002  0.001 -0.001 -0.001 -0.002 -0.002  0.     0.001  0.001
  0.003 -0.002 -0.001 -0.     0.002  0.002  0.    -0.001 -0.002 -0.001]

Formato da matriz X (frases x dimensões): (36, 100)

Vetor da palavra 'feliz':
[-0.006  0.003  0.006 -0.007  0.001 -0.008  0.007 -0.007  0.001 -0.01
 -0.009 -0.006 -0.01   0.004 -0.     0.004 -0.005  0.007 -0.001 -0.005
 -0.003 -0.009  0.001  0.001 

## Treinamento e avaliação do modelo Naive Bayes

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)  # Divide os dados em 70% treino e 30% teste, com semente fixa para reprodutibilidade
model = GaussianNB()  # Cria o modelo Naive Bayes Gaussiano, ideal para dados contínuos como vetores Word2Vec
model.fit(X_train, y_train)  # Treina o modelo com os dados de treino
y_pred = model.predict(X_test)  # Realiza a predição nos dados de teste com o modelo treinado

print("Acurácia:", accuracy_score(y_test, y_pred))  # Exibe a acurácia do modelo (percentual de acertos)
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))  # Exibe a matriz de confusão (comparação entre classes reais e previstas)
print("Relatório:\n", classification_report(y_test, y_pred))  # Gera o relatório com precision, recall e f1-score para cada classe

Acurácia: 0.18181818181818182
Matriz de Confusão:
 [[0 3]
 [6 2]]
Relatório:
               precision    recall  f1-score   support

       Feliz       0.00      0.00      0.00         3
      Triste       0.40      0.25      0.31         8

    accuracy                           0.18        11
   macro avg       0.20      0.12      0.15        11
weighted avg       0.29      0.18      0.22        11



## Predição com frases novas (teste.txt)

In [38]:
with open("teste.txt", "r", encoding="utf-8") as f:  # Abre o arquivo com frases de teste (uma por linha)
    novas_frases = [linha.split(',')[0].strip("' ") for linha in f.readlines()]  # Lê cada linha e remove aspas ou espaços extras

def vetorizacao_nova(frase):  # Função para limpar e vetorizar uma frase nova
    tokens = limpar_texto(frase)  # Aplica a limpeza e tokenização
    return vetor_frase(tokens)  # Converte a frase em vetor médio usando o modelo Word2Vec

vetores_novos = [vetorizacao_nova(f) for f in novas_frases]  # Aplica a vetorização para todas as frases novas
pred_novos = model.predict(vetores_novos)  # Realiza a predição dos sentimentos com o modelo treinado

for frase, pred in zip(novas_frases, pred_novos):  # Mostra cada frase com o respectivo sentimento previsto
    print(f"Frase: {frase}\n→ Sentimento previsto: {pred}\n")

Frase: Por vezes o sorriso que você vê no meu rosto esconde a tristeza do meu coração!
→ Sentimento previsto: Feliz

Frase: Pare de reclamar! Há pessoas com problemas maiores acredite!
→ Sentimento previsto: Triste

Frase: A gratidão é a memória do coração.
→ Sentimento previsto: Feliz

Frase: Não ouse me chamar de fraco você não sabe o que eu venho enfrentando.
→ Sentimento previsto: Triste

Frase: Obrigado Deus pelo dia de hoje!
→ Sentimento previsto: Triste



## Balanceamento de classes com SMOTE

In [39]:
# Codificar os rótulos para SMOTE
le = LabelEncoder()  # Cria um codificador para transformar os rótulos de texto em valores numéricos
y_encoded = le.fit_transform(y)  # Aplica a codificação (ex: 'Feliz' → 0, 'Triste' → 1)

# Aplicar SMOTE (k=1 pois a base é pequena)
smote = SMOTE(random_state=42, k_neighbors=1)  # Instancia o SMOTE com 1 vizinho, adequado para conjuntos pequenos
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)  # Aplica o SMOTE gerando exemplos sintéticos para balancear as classes

# Visualizar nova distribuição
Counter(le.inverse_transform(y_resampled))  # Mostra a contagem de exemplos por classe após o balanceamento (com os rótulos originais)

Counter({'Feliz': 19, 'Triste': 19})

## Novo treinamento e avaliação com dados balanceados

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)  # Divide os dados balanceados em treino (70%) e teste (30%)
model = GaussianNB()  # Cria uma instância do classificador Naive Bayes Gaussiano
model.fit(X_train, y_train)  # Treina o modelo com os dados de treino balanceados
y_pred = model.predict(X_test)  # Realiza a predição com o modelo treinado sobre os dados de teste

print("Acurácia:", accuracy_score(y_test, y_pred))  # Exibe a acurácia do modelo com os dados balanceados
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))  # Exibe a matriz de confusão com os dados balanceados
print("Relatório de Classificação:\n", classification_report(y_test, y_pred, target_names=le.classes_))  # Mostra o relatório com métricas para cada classe usando os nomes originais

Acurácia: 0.5833333333333334
Matriz de Confusão:
 [[4 2]
 [3 3]]
Relatório de Classificação:
               precision    recall  f1-score   support

       Feliz       0.57      0.67      0.62         6
      Triste       0.60      0.50      0.55         6

    accuracy                           0.58        12
   macro avg       0.59      0.58      0.58        12
weighted avg       0.59      0.58      0.58        12



## Predição com frases novas (com modelo balanceado)

In [43]:
with open("teste.txt", "r", encoding="utf-8") as f:  # Abre o arquivo contendo frases para teste
    novas_frases = [linha.split(',')[0].strip("' ") for linha in f.readlines()]  # Lê cada linha, remove aspas e espaços extras

vetores_novos = [vetorizacao_nova(f) for f in novas_frases]  # Aplica a vetorização (Word2Vec) para cada nova frase
pred_novos = model.predict(vetores_novos)  # Usa o modelo treinado para prever o sentimento das novas frases

for frase, pred in zip(novas_frases, pred_novos):  # Itera sobre cada frase e seu sentimento previsto
    print(f"Frase: {frase}\n→ Sentimento previsto: {le.inverse_transform([pred])[0]}\n")  # Exibe a frase original e o rótulo previsto (convertido de volta para texto)

Frase: Por vezes o sorriso que você vê no meu rosto esconde a tristeza do meu coração!
→ Sentimento previsto: Triste

Frase: Pare de reclamar! Há pessoas com problemas maiores acredite!
→ Sentimento previsto: Triste

Frase: A gratidão é a memória do coração.
→ Sentimento previsto: Feliz

Frase: Não ouse me chamar de fraco você não sabe o que eu venho enfrentando.
→ Sentimento previsto: Triste

Frase: Obrigado Deus pelo dia de hoje!
→ Sentimento previsto: Feliz

